# भाग 1: स्थानीय रूप से ग्रिड नोड लॉन्च करें


इस ट्यूटोरियल में, आप सीखेंगे कि ग्रिड नोड को स्थानीय मशीन में कैसे तैनात किया जाए और फिर PySyft का उपयोग करके इसके साथ सहभागिता करें।

_चेतावनी: ग्रिड नोड ऑनलाइन डेटासेट प्रकाशित करते हैं और केवल प्रायोगिक उपयोग के लिए हैं। अपने जोखिम पर नोड्स तैनात करें। किसी भी डेटा / मॉडल के साथ OpenGrid का उपयोग न करें जिसे आप निजी रखना चाहते हैं।_


नोड को स्थानीय रूप से चलाने के लिए आपको केवल एक एप्लिकेशन चलाना होगा और फिर ग्रिड वर्कर के माध्यम से इसके साथ संचार करना शुरू करना होगा। 
इस ट्यूटोरियल में हम उपलब्ध [वेबसोकेट ऐप](https://github.com/OpenMined/PyGrid/tree/dev/app/websocket) का उपयोग करेंगे.


### चरण 1: रिपॉजिटरी डाउनलोड करें

```bash
git clone https://github.com/OpenMined/PyGrid/
```


### चरण 2: निर्भरताएँ डाउनलोड करें


आपको ऐप निर्भरता स्थापित करने की आवश्यकता होगी। हम लाइब्रेरी संस्करणों के साथ समस्याओं से बचने के लिए एक स्वतंत्र [conda environment](https://docs.conda.io/projects/conda/en/latest/user-guide/concepts/environments.html) स्थापित करने की सलाह देते हैं।

आप चलाकर निर्भरता स्थापित कर सकते हैं:

```bash
cd PyGrid/app/websocket/
pip install -r requirements.txt
```

### चरण 3: ऐप शुरू करें


फिर, एप्लिकेशन शुरू करने के लिए, बस `websocket_app` स्क्रिप्ट चलाएँ।

```bash
cd PyGrid/app/websocket
python websocket_app.py --db_url=<redis_url, None> --id=<worker_id> --port=<port number>
```

यह आईडी पर `<worker_id>`  पते के बराबर आईडी के साथ ऐप शुरू करेगा: `http://0.0.0.0/<port_number>`. `--db_url`  एक REDIS डेटाबेस शुरू करता है। 
आप बिना db वाले वेबसोकेट का भी उपयोग कर सकते हैं।


यह जाँचने के लिए कि इस ऐप को चलाते समय आप किन अन्य तर्कों का उपयोग कर सकते हैं, चलाएँ:

```bash
python websocket_app.py --help
```


आइए एक कार्यकर्ता को `bob` आइडी के बराबर पोर्ट `3000` पर शुरू करें

```bash
python websocket_app.py --db_url=redis:///redis:6379 --id=bob --port=3000
```

महान। यदि ऐप सफलतापूर्वक शुरू हो गया तो स्क्रिप्ट को चालू रखना चाहिए।


### चरण 4: एक ग्रिड कार्यकर्ता बनाना और संचार शुरू करना

आइए इस ऐप से बात करने के लिए एक WebsocketGridWorker को इंस्टेंट करें।

In [1]:
# General dependencies
import torch as th
import syft as sy
from syft.workers.node_client import NodeClient

hook = sy.TorchHook(th)

In [2]:
# WARNING: We should use the same id and port as the one used to start the app!!!
worker = NodeClient(hook, "ws://localhost:3000")

worker.id

'Bob'

### चरण 5: सामान्य की तरह PySyft का उपयोग करें

अब आप बस आपके द्वारा बनाए गए कार्यकर्ता का उपयोग कर सकते हैं जैसे कि आप किसी भी अन्य सामान्य PySyft कार्यकर्ता। PySyft कैसे काम करता है, इसके बारे में अधिक जानने के लिए, कृपया PySyft ट्यूटोरियल देखें: https://github.com/OpenMined/PySyft/tree/dev/examples/tutorials

In [3]:
x = th.tensor([1,2,3,4]).send(worker)
x

(Wrapper)>[PointerTensor | me:57749333399 -> Bob:87480523874]

In [4]:
y = x + x
y

(Wrapper)>[PointerTensor | me:53206369237 -> Bob:35008750669]

In [5]:
y.get()

tensor([2, 4, 6, 8])